**The script is lightly test with demo index, more testing is needed before running as a Job and source index has to be finalised**

In [1]:
!pip install elasticsearch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.9/387.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.8 MB/s eta 0:00:00


In [2]:
from datetime import datetime, timedelta
from elasticsearch import Elasticsearch

In [9]:
# Elasticsearch connection configuration
es = Elasticsearch(["http://35.177.112.190:9200"], http_auth=("new_admin", "new_password"))

# Define the source and destination indices
source_index = 'mw-index-ftfarms'
destination_index = 'mw-index-demo'

# Calculate the timestamp range for the last 24 hours
end_time = datetime.now()
start_time = end_time - timedelta(hours=24)

<ipython-input-9-77eaf8dda1f2>:2: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(["http://35.177.112.190:9200"], http_auth=("new_admin", "new_password"))


In [10]:
def get_query():
  query = {
      "query": {
          "bool": {
              "filter": [
                  {"range": {"@timestamp": {"gte": start_time, "lte": end_time}}},
              ]
          }
      },
      "size": 1000  # Adjust the batch size as per your needs
  }
  return query

In [11]:
def anonymise_and_push(scroll_results):
  scroll_id = scroll_results['_scroll_id']

  while len(scroll_results['hits']['hits']) > 0:
      # Process each document and update the 'Recipe' field
      for hit in scroll_results['hits']['hits']:
          document_id = hit['_id']
          document_source = hit['_source']
          
          # Update the 'Recipe' field
          if document_source['FLAG'] == 'START' or document_source['FLAG'] == 'END':
            document_source['Recipe'] = 'PAL-Demo-'+document_source['Recipe']
            document_source['Line_Leader'] = 'PAL-Demo-'+document_source['Line_Leader']

          elif document_source['FLAG'] == 'SCALE':
            document_source['Op_Name'] = 'PAL-Demo-'+document_source['Op_Name']
          
          ## what fields to change and when to change has to be discussed first

          # Push the updated document to the destination index
          es.index(index=destination_index, id=document_id, body=document_source)
      
      # Scroll to the next batch of results
      scroll_results = es.scroll(scroll_id=scroll_id, scroll='1m')
      scroll_id = scroll_results['_scroll_id']

  # Clear the scroll context
  es.clear_scroll(scroll_id=scroll_id)


In [12]:
# Elasticsearch query to retrieve documents within the specified timestamp range

query = get_query()

# Scroll through the search results and update the 'Recipe' field
scroll_results = es.search(index=source_index, body=query, scroll='1m')

ann_results = anonymise_and_push(scroll_results)

<ipython-input-12-ea8a84421e0f>:6: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  scroll_results = es.search(index=source_index, body=query, scroll='1m')
<ipython-input-12-ea8a84421e0f>:6: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  scroll_results = es.search(index=source_index, body=query, scroll='1m')
<ipython-input-11-c583cb38c635>:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.index(index=destination_index, id=document_id, body=document_source)
<ipython-input-11-c583cb38c635>:21: ElasticsearchWarning: Elastic

NotFoundError: ignored